# prepare ljspeech

In [2]:
!pwd

/mnt/workspace/chunkspeech


In [5]:
# parepare models from git 
!git clone "https://github.com/NVIDIA/DeepLearningExamples.git"
!cp -r ./DeepLearningExamples/PyTorch/SpeechRecognition/Jasper ./
#!cp -r ./DeepLearningExamples/PyTorch/SpeechSynthesis/Tacotron2 ./

# download checkpoint
!wget -N  https://raw.githubusercontent.com/yhgon/colab_utils/master/gfile.py  
!python gfile.py -u 'https://drive.google.com/file/d/1u2mtrVis5W4PpX3ajHcI01ZiXNpVItc_/view?usp=sharing' -f 'jasper_fp16.pt'
## download pre-trained Tacotron2 FP16 model
#!curl -L https://developer.nvidia.com/joc-tacotron2-fp16-pyt-20190306 > JoC_Tacotron2_FP16_PyT_20190306
## download pre-trained WaveGlow FP16 model
#!curl -L https://developer.nvidia.com/joc-waveglow-fp16-pyt-20190306 > JoC_WaveGlow_FP16_PyT_20190306

# download modules
#!git clone https://github.com/NVIDIA/dllogger.git
#!cp -r dllogger/dllogger ./Tacotron2/

fatal: destination path 'DeepLearningExamples' already exists and is not an empty directory.
--2020-02-27 14:47:25--  https://raw.githubusercontent.com/yhgon/colab_utils/master/gfile.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.64.133, 151.101.128.133, 151.101.192.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3489 (3.4K) [text/plain]
Saving to: ‘gfile.py’

gfile.py            100%[===================>]   3.41K  --.-KB/s    in 0.002s  

Last-modified header missing -- time-stamps turned off.
2020-02-27 14:47:25 (1.45 MB/s) - ‘gfile.py’ saved [3489/3489]

It took  1min 46.08sec to download 2.5 GB jasper_fp16.pt 
fatal: destination path 'dllogger' already exists and is not an empty directory.


## modify script

In [6]:
%%file ./Jasper/utils/convert_librispeech.py

#!/usr/bin/env python
import argparse
import os
import glob
import multiprocessing
import json

import pandas as pd

from preprocessing_utils import parallel_preprocess

parser = argparse.ArgumentParser(description='Preprocess LibriSpeech.')
parser.add_argument('--input_dir', type=str, required=True,
                    help='LibriSpeech collection input dir')
parser.add_argument('--dest_dir', type=str, required=True,
                    help='Output dir')
parser.add_argument('--output_json', type=str, default='./',
                    help='name of the output json file.')
parser.add_argument('-s','--speed', type=float, nargs='*',
                    help='Speed perturbation ratio')
parser.add_argument('--target_sr', type=int, default=None,
                    help='Target sample rate. '
                         'defaults to the input sample rate')
parser.add_argument('--overwrite', action='store_true',
                    help='Overwrite file if exists')
parser.add_argument('--parallel', type=int, default=multiprocessing.cpu_count(),
                    help='Number of threads to use when processing audio files')
args = parser.parse_args()

args.input_dir = args.input_dir.rstrip('/')
args.dest_dir = args.dest_dir.rstrip('/')

import glob
import os


# No Script Version
def search_wav(input_dir):
    wav_files = glob.glob(os.path.join(input_dir, '**', '*.wav'),
                          recursive=True)
    # print(wav_files)
    input_data = []
    wav_files.sort()
    for wav_file in wav_files:
        rel_path = os.path.relpath(wav_file, input_dir)
        a = {'input_relpath': '{}'.format(os.path.dirname(rel_path)),
             'input_fname': os.path.basename(wav_file),
             'transcript': 'NO_SCRIPT'}

        input_data.append( a )
    # print(input_data)
    return input_data


# Each TrainScript in here
def build_input_arr(input_dir):
    txt_files = glob.glob(os.path.join(input_dir, '**', '*.trans.txt'),
                          recursive=True)
    input_data = []
    txt_files.sort()
    for txt_file in txt_files:
        rel_path = os.path.relpath(txt_file, input_dir)
        with open(txt_file) as fp:
            for line in fp:
                fname, _, transcript = line.partition(' ')
                input_data.append(dict(input_relpath=os.path.dirname(rel_path),
                                       input_fname=fname+'.flac',
                                       transcript=transcript))
    # print(input_data)
    return input_data


def search_script_line(input_dir):
    train_script = open(input_dir, 'r')
    lines = []
    result_lines = []
    while True:
        line = train_script.readline()
        if not line:
            break
        lines.append(line)

    for line in lines:
        line = line.split('|')
        del line[-1]
        result_lines.append(line)
    return result_lines


# LJSpeech scripts is csv type
def search_wav_metadata(input_dir):
    wav_files = glob.glob(os.path.join(input_dir, '**', '*.wav'),  recursive=True)
    scripts = glob.glob(os.path.join(input_dir, '**', '*.csv'), recursive=True)
    script_lines = search_script_line(scripts[0])
    wav_files.sort()
    input_data = []
    for idx, wav_file in enumerate(wav_files):
        rel_path = os.path.relpath(wav_file, input_dir)
        a = {'input_relpath': '{}'.format(os.path.dirname(rel_path)),
             'input_fname': os.path.basename(wav_file),
             'transcript': script_lines[idx][1]}
        input_data.append(a)
    # print(input_data)


def build_input_arr_all_sh(input_dir):
    scripts = glob.glob(os.path.join(input_dir, '**', '*.csv'), recursive=True)
    txt_files = glob.glob(os.path.join(input_dir, '**', '*.trans.txt'),
                          recursive=True)
    if len(scripts) == 0 and len(txt_files) == 0:
        print("no scripts in here")
        return search_wav(input_dir)
    elif len(scripts) == 1:
        print("metadata script in here")
        search_wav_metadata(input_dir)
    else:
        print("each scripts in here")
        return build_input_arr(input_dir)

    
print("[%s] Scaning input dir..." % args.output_json)
dataset = build_input_arr_all_sh(input_dir=args.input_dir)

print("[%s] Converting audio files..." % args.output_json)
dataset = parallel_preprocess(dataset=dataset,
                              input_dir=args.input_dir,
                              dest_dir=args.dest_dir,
                              target_sr=args.target_sr,
                              speed=args.speed,
                              overwrite=args.overwrite,
                              parallel=args.parallel)

print("[%s] Generating json..." % args.output_json)
df = pd.DataFrame(dataset, dtype=object)

# Save json with python. df.to_json() produces back slashed in file paths
dataset = df.to_dict(orient='records')
with open(args.output_json, 'w') as fp:
    json.dump(dataset, fp, indent=2)

Overwriting ./Jasper/utils/convert_librispeech.py


# denoising dataset

In [7]:
!python gfile.py -u 'https://drive.google.com/file/d/1QfcuxcSyJnb8V6KW1ioGAg_J77UxzixS/view?usp=sharing' -f 'ljspeech_deno_cycle1.zip'

It took  3min 16.62sec to download 2.9 GB ljspeech_deno_cycle1.zip 


In [8]:
%%time
!unzip ./ljspeech_deno_cycle1.zip

Archive:  ./ljspeech_deno_cycle1.zip
   creating: ljspeech_deno_cycle1/
   creating: ljspeech_deno_cycle1/LJSpeech-1.1/
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/metadata.csv  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/README  
   creating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0001.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0002.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0003.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0004.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0005.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0006.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0007.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0008.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0009.wav  
  inflating: ljspeech_deno_cycle1/LJSpeech-1.1/wavs/LJ001-0010.wav  
  inflating: ljspee

## split base on silence

In [9]:
%%file ./chunk_split.py

from IPython.display import Image
import librosa
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import pandas as pd
import soundfile as sf 
import librosa.display
import glob
import os
from tqdm  import tqdm
#import natsort 
import multiprocessing


    
def data_split_step_v2(input_file, output_path, sr=22050):
    filename = os.path.basename(input_file)
    data, sr = librosa.load(input_file , sr=sr )
    data_deno = custom_denoising(data, sr=sr )
    _, bound, _ = chunk_get_silence_v2(data_deno, sr=sr, chunk_top_db=60, chunk_frame_length=512, chunk_hop_length=32, vodur=0.24, sidur=0.1 )
    chunk_voice(data, bound, 22050, filename, output_path)
    
def chunk_get_silence_v2(data , sr=16000, chunk_top_db=60, chunk_frame_length=512, chunk_hop_length=32, vodur=0.24, sidur=0.1):
    y_result_level_1  = librosa.effects.split(y=data, top_db=chunk_top_db, frame_length=chunk_frame_length, hop_length=chunk_hop_length)  
    data = np.array( y_result_level_1 )
    dataset = pd.DataFrame({'vo_s': data[:, 0], 'vo_e': data[:, 1]})
    dataset['vo_d']     = dataset['vo_e'] - dataset['vo_s']
    dataset['vo_s_sec'] = dataset['vo_s']/sr
    dataset['vo_e_sec'] = dataset['vo_e']/sr  
    dataset['vo_d_sec'] = dataset['vo_d']/sr 
    new_data = dataset.query('vo_d_sec > {}  '.format(vodur)  ).reset_index(drop=True) # 0.3/758, 0.4/743,  0.5/720,  0.6/704 , 0.7/687 
    new_data['si_s']     = new_data['vo_e'].shift(periods=1, fill_value=0)+1
    new_data['si_e']     = new_data['vo_s']-1
    new_data['si_d']     = new_data['si_e'] - new_data['si_s']
    new_data['si_s_sec'] = new_data['si_s']/sr
    new_data['si_e_sec'] = new_data['si_e']/sr  
    new_data['si_d_sec'] = new_data['si_d']/sr 
    new_data2 = new_data.query('si_d_sec > {}'.format(sidur)  ).reset_index(drop=True)
    new_data2.drop(['vo_s','vo_e','vo_d','vo_s_sec','vo_e_sec','vo_d_sec','si_s_sec', 'si_d','si_e_sec'],axis=1)
    new_data2['vo_s'] = new_data2['si_e']+1
    new_data2['vo_e'] = new_data2['si_s'].shift(periods=-1,  fill_value=None)-1
    new_data2['vo_d'] = new_data2['vo_e'] - new_data2['vo_s']
    new_data2['vo_s_sec'] = new_data2['vo_s']/sr
    new_data2['vo_e_sec'] = new_data2['vo_e']/sr  
    new_data2['vo_d_sec'] = new_data2['vo_d']/sr
    return dataset, new_data, new_data2
     
def chunk_voice(data, bound, sr, filename, output_path):
    voice = bound[['vo_s', 'vo_e']]
    voice = voice.values.tolist()
    head = filename.split('.')[0]
    for idx, line in enumerate(voice):
        startpoint = line[0]/sr
        endpoint = line[1]/sr
        y_split = data[int(line[0]):int(line[1])]
        y_split_centerpad = librosa.util.pad_center(y_split, int(sr * 0.5) + len(y_split) ) 
        # librosa.output.write_wav('{}/{}-{}.wav'.format(output_path,head,str(idx).zfill(3)), y_split, sr)
        sf.write('{}/{}-{}.wav'.format(output_path,head,str(idx).zfill(3)), y_split_centerpad, sr)

def data_split_step(input_file, output_path, args):
    
    filename = os.path.basename(input_file)
    data, sr = librosa.load(input_file , sr=int(args.sr) )
    _, bound, _ = chunk_get_silence_v2(data, 
                                       sr=sr , 
                                       chunk_top_db= int(args.chunk_top_db), 
                                       chunk_frame_length= int(args.chunk_frame_length), 
                                       chunk_hop_length= int(args.chunk_hop_length) , 
                                       vodur= float(args.vodur), 
                                       sidur=float(args.sidur) )
    chunk_voice(data, bound, args.sr, filename, output_path)

    
def main(args ):
    import os

    
    
    process_count = multiprocessing.cpu_count()
    max_cores =  args.max_cores 
    process_count_submit =  min(process_count, max_cores ) 
    print("process_count : {}".format(process_count_submit))
    
    print(args)
    
    input_dir  = args.input_dir
    output_dir = args.output_dir
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    wav_files = sorted(glob.glob(os.path.join(input_dir, '**', '*.wav'), recursive=True))
    iter_files = []
    for i,wav in enumerate(wav_files):
        if i >=  args.max_files  :
            break        
        iter_tuple = (wav, output_dir, args)
        iter_files.append(iter_tuple)
        i=i+1
    print("DEBUG : ready for {} files with {} cores".format(i, process_count_submit)  )
    print(len(iter_files))
    pool = multiprocessing.Pool(processes=int(process_count_submit) )
    pool.starmap(data_split_step, iter_files )
    pool.close()
    pool.join()
    
    
if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_dir',          required=True,                help=' ')
    parser.add_argument('--output_dir',         required=True,                help=' ')    
    parser.add_argument('--max_files',          default=100000000,  type=int, help=' ')    
    parser.add_argument('--max_cores',          default=20,     type=int, help=' ')     
    parser.add_argument('--sr',                 default=22050,  type=int, help=' ')
    parser.add_argument('--chunk_top_db',       default=60, type=int,    help='librosa db')
    parser.add_argument('--chunk_frame_length', default=512, type=int,    help='librosa frame  ')
    parser.add_argument('--chunk_hop_length',   default=32,  type=int,    help=' librosa chunk hop ')
    parser.add_argument('--vodur',              default=0.1, type=float,  help='vocal duration ')
    parser.add_argument('--sidur',              default=0.1, type=float,  help='silence duration ') 
        
    args = parser.parse_args()    
    
    main(args  )
    


Writing ./chunk_split.py


In [14]:
import scipy.io.wavfile as wav
import glob
import os 


def len_wav(file_path):
    import scipy.io.wavfile as wav
    import glob
    import os     
    (source_rate, source_sig) = wav.read(file_path)
    duration_seconds = len(source_sig) / float(source_rate)
    return "{:4.2f}".format(duration_seconds)


def wav_list(work_dir, file_ext='txt'):
    import glob
    import os     
    list_files = []
    targets = os.path.join(  work_dir , '*.{}'.format(file_ext)  ) 
    for file in glob.glob(targets ):
        list_files.append(file)
    return list_files   

def wav_len_dir(work_dir):
    import numpy as np
    lists_wav = wav_list(work_dir, 'wav')
    lists_dur =[]
    for i,v in enumerate(lists_wav):
        dur = len_wav(v)
        #print(i,v,dur)
        
        lists_dur.append(dur)
    return np.array(lists_dur).astype(float)

In [13]:

input_dir   = './ljspeech_deno_cycle1/LJSpeech-1.1/wavs'
output_dir  = './ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1'
%time !python ./chunk_split.py    \
 --input_dir   {input_dir}  \
 --output_dir  {output_dir} \
 --max_files 140000 --max_cores 80 --sr 22050 \
 --chunk_top_db 58 \
 --sidur 0.7 \
 --vodur 0.4 
 
!ls  {input_dir} | wc -l
!ls  {output_dir} | wc -l




process_count : 80
Namespace(chunk_frame_length=512, chunk_hop_length=32, chunk_top_db=58, input_dir='./ljspeech_deno_cycle1/LJSpeech-1.1/wavs', max_cores=80, max_files=140000, output_dir='./ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1', sidur=0.7, sr=22050, vodur=0.4)
DEBUG : ready for 13100 files with 80 cores
13100
CPU times: user 28 s, sys: 11.4 s, total: 39.4 s
Wall time: 29min 15s
13100
30412


NameError: name 'np' is not defined

In [15]:
x = wav_len_dir(input_dir)
y = wav_len_dir(output_dir)

num_bin=100
import numpy as np
import matplotlib.pyplot as plt
n, bins, patches = plt.hist(x,num_bin, alpha=0.75)
n, bins, patches = plt.hist(y,num_bin, alpha=0.75)
plt.title('Histogram of wav duration of LJSpeech(chunk)')
plt.xlabel('duration(sec)')
plt.show()

KeyboardInterrupt: 

## manifest for jasper

In [16]:
!mkdir ./ljspeech_deno_cycle1/LJSpeech-1.1/wavs_jasper

In [17]:
!python ./Jasper/utils/convert_librispeech.py \
    --input_dir ./ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1 \
    --dest_dir  ./ljspeech_deno_cycle1/LJSpeech-1.1/wavs_jasper \
    --target_sr 16000 \
    --output_json ./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json

[./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json] Scaning input dir...
no scripts in here
[./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json] Converting audio files...
100%|█████████████████████████████████████| 30412/30412 [16:52<00:00, 30.04it/s]
[./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json] Generating json...


## inference

In [18]:
%%time
!python ./Jasper/inference.py  \
 --batch_size 10 \
 --dataset_dir ./ljspeech_deno_cycle1/LJSpeech-1.1 \
 --val_manifest ./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json \
 --model_toml ./Jasper/configs/jasper10x5dr_sp_offline_specaugment.toml \
 --seed 0 --ckpt ./jasper_fp16.pt \
 --fp16 \
 --save_prediction ./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json.predictions.txt

Arguments:
	     batch_size : 10
	           ckpt : ./jasper_fp16.pt
	cudnn_benchmark : False
	    dataset_dir : ./ljspeech_deno_cycle1/LJSpeech-1.1
	   export_model : False
	           fp16 : True
	     local_rank : None
	 logits_save_to : None
	    masked_fill : None
	   max_duration : None
	     model_toml : ./Jasper/configs/jasper10x5dr_sp_offline_specaugment.toml
	     output_dir : results/
	         pad_to : None
	       pyt_fp16 : False
	save_prediction : ./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json.predictions.txt
	           seed : 0
	          steps : None
	   val_manifest : ./ljspeech_deno_cycle1/LJSpeech-1.1/ljspeech_split_try1.json
	            wav : None
CUDNN BENCHMARK  False
=== model_config ===
Arguments:
	   encoder : {'activation': 'relu', 'convmask': True}
	     input : {'normalize': 'per_feature', 'sample_rate': 16000, 'window_size': 0.02, 'window_stride': 0.01, 'window': 'hann', 'features': 64, 'n_fft': 512, 'frame_splicing': 1, 'dither': 1e-05, 'f

## text processing

In [19]:
# load datasets

def get_filelist_manifest(path, filename):
    import json    
    import os
    full_file_name_input = os.path.join(path, filename)
    filelists = []
    with open(full_file_name_input ) as ref:
        ref_data = json.load(ref)
        for line in ref_data:
            file_full = line['files'][0]['fname']
            file_body = os.path.split(file_full)[-1]
            filelists.append(file_body)
        ref.close()
    return filelists

def read_prediction(path, filename):
    import json    
    import os
    full_file_name_input = os.path.join(path, filename)   
    prediction = []
    with open(full_file_name_input ) as ref_data:
        for line in ref_data:
            prediction.append(line)
        ref_data.close()
    return prediction

def len_wav(file_path):
    import scipy.io.wavfile as wav
    import glob
    import os     
    (source_rate, source_sig) = wav.read(file_path)
    duration_seconds = len(source_sig) / float(source_rate)
    return "{:4.2f}".format(duration_seconds)

def write_wav_text_filelist_from_jasper(
    manifest_dir, manifest_file, pred_dir, pred_file,  output_dir , output_filename, 
    dataset_dir='', ljs_dir='LJSpeech-1.1', wav_dir='wavs', 
    threshold = 0.8, 
    chars = 8 ):
    
    import os    
    manifest_list = get_filelist_manifest(manifest_dir, manifest_file)
    predict_list  = read_prediction(pred_dir, pred_file)
    
    full_file_name_output = os.path.join(output_dir, output_filename) 
    
    full_data_dir = os.path.join(dataset_dir, ljs_dir, wav_dir) 
    
    output = open(full_file_name_output, 'w')
    i=0
    for file, prediction in zip(manifest_list, predict_list):
        full_filename = os.path.join(full_data_dir, file)
        wav_len  = len_wav(full_filename)
        char_len = len(prediction)
        if (float(wav_len) > threshold) and ( char_len >= chars )  : 
            output.write('{}|{}'.format(  full_filename, prediction))
            i=i+1
    output.close()    
    print(i)

In [23]:
manifest_dir  =  './ljspeech_deno_cycle1/LJSpeech-1.1'
pred_dir      =  './ljspeech_deno_cycle1/LJSpeech-1.1'
output_dir    =  './ljspeech_deno_cycle1/LJSpeech-1.1'

manifest_file   =  'ljspeech_split_try1.json'
pred_file       =  'ljspeech_split_try1.json.predictions.txt'
output_filename =  'ljscs_full.txt'


In [22]:
!pwd
!ls .

/mnt/workspace/chunkspeech
DeepLearningExamples  dllogger		   ljspeech_deno_cycle1
Jasper		      gfile.py		   ljspeech_deno_cycle1.zip
Tacotron2	      jasper_fp16.pt
chunk_split.py	      ljscs_prepare.ipynb


In [24]:
minifest_list = get_filelist_manifest(manifest_dir , manifest_file)
predict_list  = read_prediction( pred_dir, pred_file)
print(len(minifest_list) , len( predict_list) )

30412 30412


In [30]:
!pwd 
!ls ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1
!ls /mnt/workspace/chunkspeech/ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1/LJ001-0001-000.wav

/mnt/workspace/chunkspeech
LJ001-0001-000.wav  LJ013-0196-001.wav	LJ024-0091-000.wav  LJ038-0005-000.wav
LJ001-0001-001.wav  LJ013-0197-000.wav	LJ024-0092-000.wav  LJ038-0005-001.wav
LJ001-0001-002.wav  LJ013-0198-000.wav	LJ024-0092-001.wav  LJ038-0006-000.wav
LJ001-0002-000.wav  LJ013-0198-001.wav	LJ024-0092-002.wav  LJ038-0007-000.wav
LJ001-0003-000.wav  LJ013-0198-002.wav	LJ024-0093-000.wav  LJ038-0008-000.wav
LJ001-0003-001.wav  LJ013-0199-000.wav	LJ024-0094-000.wav  LJ038-0008-001.wav
LJ001-0003-002.wav  LJ013-0199-001.wav	LJ024-0094-001.wav  LJ038-0008-002.wav
LJ001-0003-003.wav  LJ013-0199-002.wav	LJ024-0094-002.wav  LJ038-0008-003.wav
LJ001-0004-000.wav  LJ013-0200-000.wav	LJ024-0095-000.wav  LJ038-0009-000.wav
LJ001-0004-001.wav  LJ013-0200-001.wav	LJ024-0096-000.wav  LJ038-0009-001.wav
LJ001-0004-002.wav  LJ013-0201-000.wav	LJ024-0096-001.wav  LJ038-0010-000.wav
LJ001-0005-000.wav  LJ013-0201-001.wav	LJ024-0096-002.wav  LJ038-0010-001.wav
LJ001-0005-001.wav  LJ013-0202-000.wa

In [31]:
outputfile = os.path.join(output_dir,output_filename)
#!rm -rf {outputfile}
write_wav_text_filelist_from_jasper(manifest_dir , manifest_file, 
                                    pred_dir, pred_file, 
                                    output_dir, output_filename ,
                                    dataset_dir = '/mnt/workspace/chunkspeech/ljspeech_deno_cycle1',
                                    ljs_dir = 'LJSpeech-1.1',
                                    wav_dir ='wavs_split_try1' )
!head -n 1 {outputfile}

29188
/mnt/workspace/chunkspeech/ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1/LJ001-0001-000.wav|printing


In [32]:
!soxi /mnt/workspace/chunkspeech/ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1/LJ001-0001-000.wav


Input File     : '/mnt/workspace/chunkspeech/ljspeech_deno_cycle1/LJSpeech-1.1/wavs_split_try1/LJ001-0001-000.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:01.20 = 26353 samples ~ 89.6361 CDDA sectors
File Size      : 52.8k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM



In [33]:
!tail -n+500 ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_full.txt   > ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_train.txt
!head -n500  ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_full.txt   > ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_test.txt

In [34]:
!cp -rf ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_train.txt  /mnt/workspace/git/2020_0227_t2/tacotron2/filelists/ljscs_train.txt
!cp -rf ./ljspeech_deno_cycle1/LJSpeech-1.1/ljscs_test.txt   /mnt/workspace/git/2020_0227_t2/tacotron2/filelists/ljscs_test.txt